In [2]:
import numpy as np
import sys
import itk
import os

cwd = os.getcwd()
cwd

'/home/will.dolan/Desktop/CTAandCTP'

In [ ]:
# Directories if not yet made
os.system('mkdir Masks')
os.system('mkdir NRRD-Files')

In [13]:
# NRRD Files
directory = (cwd + '/NRRD-Files/CC012-CTP-Frames/')

# Saved NRRD Files
directory2 = (cwd + '/NRRD-Files/CC012-Frames-Reg/')


pic_folder = directory
pic_folder = os.listdir(pic_folder)


# Mask Creation and Location
MASK_DIRECTORY = (cwd + '/Masks/')
mask_num = len(os.listdir(MASK_DIRECTORY)) + 1
os.system('~/src/ITKTubeTK-Release/bin/ImageMath ' + directory + pic_folder[0] + ' -t 100 5000 1 0 -M 1 10 1 0 -W 0 ' + MASK_DIRECTORY + 'mask' + str(mask_num) + '.mha')
print('mask' + str(mask_num))

# ITKTubeTK binary location
tubetk = '/home/will.dolan/src/ITKTubeTK-Release/bin/RegisterImages'


Dimension = 3
PixelType = itk.ctype('float')
ImageType = itk.Image[PixelType, Dimension]

out = ImageType.New()

start = itk.Index[Dimension]()
start[0] = 0  
start[1] = 0  
start[2] = 0  

pic_folder.sort()

img0 = itk.imread(directory + pic_folder[0])

size = itk.Size[Dimension]()
size[0] = itk.size(img0)[0]
size[1] = itk.size(img0)[1]
size[2] = itk.size(img0)[2]

num_images = len(pic_folder)

imgdatasize = np.zeros(4, dtype=int)
imgdatasize[0] = num_images
imgdatasize[1] = size[2]
imgdatasize[2] = size[1]
imgdatasize[3] = size[0]
imgdata = np.zeros(imgdatasize)
imgdata.shape
        
for image in range(num_images):
    if image != 0:
        oscmd = (tubetk + ' ' + directory + pic_folder[0] + ' ' + directory + pic_folder[image] + ' --resampledImage ' + directory2 + pic_folder[image] + ' --rigidMaxIterations 3000 --registration Rigid --metric MeanSqrd --fixedImageMask ' + MASK_DIRECTORY + 'mask5.mha --sampleFromOverlap')
        os.system(oscmd)
    else:
        oscmd = ('cp ' + directory + pic_folder[image] + ' ' + directory2 + pic_folder[image])
        os.system(oscmd)
    img = itk.imread(directory2 + pic_folder[image])
    imgdata[image,:,:,:] = itk.GetArrayFromImage(img)

    percent = (image + 1) / num_images * 100
    print(str(round(percent)) + '%')
    
print('Done')    

mask13
3%
5%
8%
10%
13%
15%
18%
21%
23%
26%
28%
31%
33%
36%
38%
41%
44%
46%
49%
51%
54%
56%
59%
62%
64%
67%
69%
72%
74%
77%
79%
82%
85%
87%
90%
92%
95%
97%
100%
Done


In [14]:
outdata = imgdata.max(0).astype('h')

out = itk.GetImageFromArray(outdata)
out.CopyInformation(img0)
final = itk.imwrite(out, (cwd + '/Composites/CC012-Max.nrrd'))

print('Done')